In [ ]:
import numpy as np
from wrapping import run_cma, simulate_single

# Common sim settings (feel free to tweak)
sim_kwargs = dict(
    N=32,
    dt=0.1,
    T=800,
    K_snap=5,
    max_abs=5.0,
    alpha=0.5,     # normalization strength in denominator
    beta_shift=0.1 # anti-shift penalty weight
)

best_K, best_score = run_cma(
    x0=np.zeros(3, dtype=np.float32),  # CMA start
    sigma0=0.7,
    popsize=256,
    bounds=(-1.0, 1.0),
    maxiter=60,
    sim_kwargs=sim_kwargs,
    verbose=True
)

print("\n=== CMA result ===")
print("Best K:", best_K)
print("Best score (single-run):", best_score)


In [ ]:
# === Minimal viz for a found K (wrapping, RK4) ===
import numpy as np
import matplotlib.pyplot as plt

def neighbors_wrapped(a: np.ndarray):
    return np.roll(a, 1), a, np.roll(a, -1)

def rhs_linear(a: np.ndarray, K: np.ndarray):
    k1, k2, k3 = K
    L, C, R = neighbors_wrapped(a)
    return k1*L + k2*C + k3*R - 0.05 *C**3

def rk4_step(a: np.ndarray, dt: float, K: np.ndarray):
    s1 = rhs_linear(a, K)
    s2 = rhs_linear(a + 0.5*dt*s1, K)
    s3 = rhs_linear(a + 0.5*dt*s2, K)
    s4 = rhs_linear(a + dt*s3, K)
    return a + (dt/6.0)*(s1 + 2*s2 + 2*s3 + s4)

def simulate_traj(
    K, N=64, dt=0.05, T=320, max_abs=5.0, a0_scale=0.05
):
    """Return trajectory (T+1, N) with wrapping; stops early on overflow."""
    K = np.asarray(K, dtype=np.float32).reshape(3,)
    a = (a0_scale * np.random.standard_normal(N)).astype(np.float32)
    traj = np.empty((T+1, N), dtype=np.float32)
    traj[0] = a
    t_stop = T
    for t in range(1, T+1):
        a = rk4_step(a, dt, K)
        if (not np.isfinite(a).all()) or np.any(np.abs(a) > max_abs):
            t_stop = t
            traj[t] = 0.0  # mark overflow moment as zeros
            traj = traj[:t+1]
            break
        traj[t] = a
    return traj, (t_stop != T)

def plot_spacetime(traj: np.ndarray, title="Space–Time evolution"):
    """imshow of time (rows) vs space (cols), plus final state line plot."""
    fig = plt.figure(figsize=(20, 10))
    ax1 = plt.subplot(1, 2, 1)
    im = ax1.imshow(traj, aspect='auto', origin='lower', cmap='viridis')
    ax1.set_xlabel('cell index')
    ax1.set_ylabel('time step')
    ax1.set_title(title)
    plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04)

    ax2 = plt.subplot(1, 2, 2)
    ax2.plot(traj[-1], lw=1.5)
    ax2.set_title('Final state')
    ax2.set_xlabel('cell index'); ax2.set_ylabel('value')
    plt.tight_layout()
    plt.show()

# ---- Example: plug your found K here ----
K_found = np.array([-0.02641137, -0.88760346,  0.02883846], dtype=np.float32)  # <- replace with your best K
traj, overflowed = simulate_traj(K_found, N=64, dt=0.1, T=800, max_abs=5.0, a0_scale=0.5)
plot_spacetime(traj, title=f"RK4 wrapping | K={K_found.tolist()} | overflow={overflowed}")